In [1]:
import datetime
import json
import os
import typing

import pandas as pd
import requests


In [ ]:
curl -v -X POST -u "ZGY0NzUxNjAtYThhMC00YWI0LWE0ZGYtNWY1YmFkYjI2ZmQz:NjFjOTgxYTYtM2M2Zi00Y2IxLTk5NTQtZDliYTdiZGM1ZmY3" -d "username=tgadfort@gmail.com&password=Charlie2022$&grant_type=password" "https://napi-v2-2-cloud-run-b3gtd5nmxq-uw.a.run.app/oauth/token"

In [ ]:
curl -v -X POST -u "ZGY0NzUxNjAtYThhMC00YWI0LWE0ZGYtNWY1YmFkYjI2ZmQz:NjFjOTgxYTYtM2M2Zi00Y2IxLTk5NTQtZDliYTdiZGM1ZmY3" -d "username=tgadfort@gmail.com&password=Charlie2022$&grant_type=password" "https://napi-v2-2-cloud-run-b3gtd5nmxq-uw.a.run.app/oauth/token"

In [1]:
import requests
baseURL = "https://napi-v2-2-cloud-run-b3gtd5nmxq-uw.a.run.app"
apikey = 'ZGY0NzUxNjAtYThhMC00YWI0LWE0ZGYtNWY1YmFkYjI2ZmQz'
apisec = 'NjFjOTgxYTYtM2M2Zi00Y2IxLTk5NTQtZDliYTdiZGM1ZmY3'
user = 'tgadfort@gmail.com'
password = 'Charlie2022$'

OAUTH_URL = f'{baseURL}/oauth/token'
params = (apikey, apisec)
data = {'username': user,
        'password': password,
        'grant_type': 'password'}
print("Getting OAuth Token ... ", end="")
response_login = requests.post(url=OAUTH_URL, auth=params, data=data, timeout=10)
print(response_login)
print(response_login.text)
tokens = json.loads(response_login.text)
#self.token = tokens['access_token']
#print(self.token)
#self.headers = {'Authorization': f"Bearer {self.token}"}


Getting OAuth Token ... <Response [200]>
{"access_token":"NDYxZDYxMGMtZGEzYi00M2Y5LWIxM2ItNWViMjZkN2QyNjVj","refresh_token":"2216bb08-9842-41cc-9e36-aa04a2123f8a","expires_in":86399,"token_type":"BearerToken","username":"tgadfort@gmail.com","first_name":"Thomas","last_name":"Gadfort","guid":"E57DF53D3DC9595AE0535403960A7DB5","catalog":"US","authCodeId":null}


NameError: name 'json' is not defined

In [3]:
response_login.text

'{"access_token":"NDYxZDYxMGMtZGEzYi00M2Y5LWIxM2ItNWViMjZkN2QyNjVj","refresh_token":"2216bb08-9842-41cc-9e36-aa04a2123f8a","expires_in":86399,"token_type":"BearerToken","username":"tgadfort@gmail.com","first_name":"Thomas","last_name":"Gadfort","guid":"E57DF53D3DC9595AE0535403960A7DB5","catalog":"US","authCodeId":null}'

In [2]:
class NapsterAPIcall():

    API_URL = 'https://api.napster.com/'
    API_VERSION = 'v2.2'
    API_KEY = 'ZGY0NzUxNjAtYThhMC00YWI0LWE0ZGYtNWY1YmFkYjI2ZmQz' # os.environ['napster_API_KEY']
    API_SECRET = 'NjFjOTgxYTYtM2M2Zi00Y2IxLTk5NTQtZDliYTdiZGM1ZmY3' # os.environ['napster_API_SECRET']
    
    user = 'tgadfort@gmail.com'
    password = 'Charlie2022$'
    # user = os.environ['napster_user']
    # password = os.environ['napster_password']
    
    headers = None
    
    def __init__(self):
        self.header = None
        print('init')
    
    def login(self):
        # Login
        OAUTH_URL =  f'{self.API_URL}oauth/token'
        params = (self.API_KEY, self.API_SECRET)
        data = {
            'username': self.user,
            'password': self.password,
            'grant_type': 'password'
            }
        response_login = requests.post(url=OAUTH_URL, auth=params, data=data)
        tokens = json.loads(response_login.text)
        self.headers = {'Authorization': f"Bearer {tokens['access_token']}"}
        
        return response_login.status_code
    

    def fetch_account_info(self):
    # Account Information
        response = requests.get(url=f'{self.API_URL}{self.API_VERSION}/me/account', headers=self.headers)
        return(response)

    def _fetch_favorite_tracks(self, offset):
    # Favorite Tracks
        params = {'limit': 200, 'offset': offset, 'filter': 'track'}
        response = requests.get(url=f'{self.API_URL}{self.API_VERSION}/me/favorites', 
                                headers=self.headers,
                                params = params)
        #file = open("favorites.json","w+")
        #file.write(response.text)
        #file.close()
        return response.status_code, json.loads(response.text)
    
    def fetch_favorites(self):
    # All Favorite Tracks
        columns = ['artist', 'title', 'streamable']
        favorites_df = pd.DataFrame(columns=columns)
        ind = 0
        while(True):
            code, favorites = self._fetch_favorite_tracks(offset=ind * 200)
            if code == 200:
                #print(favorites_tracks)
                favorites_tracks = favorites['favorites']['data']['tracks']
                for item in favorites_tracks:
                    title, artist, streamable = self.extract_from_list(item)
                    favorites_df = favorites_df.append(pd.DataFrame(data=[[artist, title, streamable]],
                                                                    columns=columns), 
                                                       ignore_index=True)
                # test for end of list
                lim = favorites['meta']['query']['limit']
                off = favorites['meta']['query']['offset']
                if favorites['meta']['totalCount'] <= (off + lim):
                    break
                
                ind += 1
            else:
                print('something went wrong!')
                return
        
        return favorites_df
    
    def extract_from_list(self, item):
        title = item['name']
        artist = item['artistName']
        streamable = item['isStreamable']
        return title, artist, streamable
    
    def fetch_from_library(self, item, subitem=None, limit=200, offset=0):
        itemstr = ''
        if subitem is None:
            itemstr = item
        else:
            itemstr = f'{item}/{subitem}'
        params = {'limit': limit, 'offset': offset}
        response = requests.get(url=f'{self.API_URL}{self.API_VERSION}/me/library/{itemstr}', 
                                headers=self.headers,
                                params=params)
        return response.status_code, json.loads(response.text)
    
    def fetch_list_of_playlists(self) -> typing.List[typing.List]:
        index = 0
        playlists_list = []
        while True:
            code, response = self.fetch_from_library(item='playlists', offset=index * 200)
            if code == 200:
                new_elements = [[item['id'], item['name']] for item in response['playlists']]
                playlists_list.extend(new_elements)
                if len(new_elements) < 200:
                    break
                index = index + 1
        return playlists_list
        
    
    def fetch_playlist(self, playlist_id) -> pd.DataFrame:
        tracks_list = []
        code, response = self.fetch_from_library(item = 'playlists', subitem = playlist_id + '/tracks')
        for item in response['tracks']:
            track_id = item['id']
            title = item['name']
            artist = item['artistName']
            tracks_list.append([track_id, title, artist])
        
        tracks_df = pd.DataFrame(tracks_list, columns=['id', 'title', 'artist'])
        return tracks_df
    
    def add_track_to_library(self, track_id):
        headers = {**self.headers, 'Content-Length': '0'}
        response = requests.post(url=f'{self.API_URL}{self.API_VERSION}/me/library/tracks?id={track_id}',
                                 headers=headers)
        
        
        

In [ ]:
def download_current_favorites(napster: NapsterAPIcall):
    favorites_df = napster.fetch_favorites()
    now = datetime.datetime.now().strftime('%Y-%m-%d')
    favorites_df.to_csv(f'favorites_{now}.csv', index = 0)

def check_all_playlists(napster: NapsterAPIcall):
    listPlaylist = napster.fetch_list_of_playlists()
    
    columns = ['playlist','artist', 'title', 'streamable']
    unstream_tracks_df = pd.DataFrame(columns=columns)
    
    for listitem in listPlaylist:
        code, response = napster.fetch_from_library(item='playlists', subitem=f'{listitem[0]}/tracks')
        for item in response['tracks']:
            title, artist, streamable = napster.extract_from_list(item)
            if not streamable:
                unstream_tracks_df = unstream_tracks_df.append(
                    pd.DataFrame(data=[[listitem[1], artist, title, streamable]],
                                 columns = columns),
                    ignore_index = True)
        #print(response)
        
    now = datetime.datetime.now().strftime('%Y-%m-%d')
    unstream_tracks_df.to_csv(f'unstreamTracks_playlists_{now}.csv', index = 0)
    return unstream_tracks_df

def add_playlist_tracks_to_library(napster: NapsterAPIcall):
    playlists = napster.fetch_list_of_playlists()
    for playlist in playlists:
        playlist_id = playlist[0]
        playlist_tracks = napster.fetch_playlist(playlist_id=playlist_id)
        for row_index, row in playlist_tracks.iterrows():
            napster.add_track_to_library(track_id=row['id'])
    

In [3]:
napsterAPI = NapsterAPIcall()
napsterAPI.login()
napsterAPI.fetch_account_info()


init


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
nonstreamabletracks = check_all_playlists(napster=napsterAPI)
print(nonstreamabletracks)

In [ ]:
display(napsterAPI.fetch_list_of_playlists())

In [ ]:
add_playlist_tracks_to_library(napster=napsterAPI)

### Auswertung Favorites

In [ ]:
favorites_df = pd.read_csv('favorites_2020-11-27.csv')

# replace additions like "(2009 Remix)" or "(Remastered 2016)"
# title all lowercase for better matching
favorites_df['title'] = favorites_df['title'].replace(' \([A-z,0-9 ]*\)', value='', regex=True)
favorites_df['title'] = favorites_df['title'].replace('[\'\?\’]', value='', regex=True)
favorites_df['title'] = favorites_df['title'].str.lower()

# fetch all tracks which are not streamble 
unst_df = favorites_df[favorites_df['streamable'] == False]
unst_df = unst_df.drop('streamable', 1)
#print(unst_df)

# fetch all substitutes aka double entries 
favorites_substitute_df = favorites_df[favorites_df.duplicated(subset=['artist', 'title'], keep='last')]
favorites_substitute_df = favorites_substitute_df[favorites_substitute_df['streamable'] == True]
#print(favorites_substitute_df)
#print(favorites_filtered_df[favorites_filtered_df['streamable'] == False])

# cross check for a list of unstreamable track which were not replaced
unst_df = unst_df.assign(Replaced=unst_df.title.isin(favorites_substitute_df.title))
print(unst_df[unst_df['Replaced'] == False])